# Main Operations with LCEL Chains

## Contents
* Chaining runnables, coercion
* Multiple chains
* Nested chains
* Fallback for chains

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
MODEL_GPT = 'gpt-4o-mini'

In [3]:
# !pip list | grep langchain
!pip list | findstr langchain

langchain                                0.3.13
langchain-chroma                         0.1.4
langchain-community                      0.3.13
langchain-core                           0.3.43
langchain-experimental                   0.3.4
langchain-google-community               2.0.7
langchain-groq                           0.2.5
langchain-openai                         0.2.14
langchain-text-splitters                 0.3.4


## Connect with LLM and start conversation with it

In [4]:
from langchain_openai import ChatOpenAI

# model = ChatOpenAI(model="gpt-3.5-turbo")
model = ChatOpenAI(model=MODEL_GPT)

## Basic chain: LLM model + prompt + output parser

In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Write one brief sentence about {politician}")

output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [6]:
chain.invoke({
    "politician": "JFK"
})

'John F. Kennedy was the 35th President of the United States, serving from 1961 until his assassination in 1963.'

## Mid-level chain: Retriever App Example

In [7]:
# !pip install docarray tiktoken

In [8]:
# !pip list | grep tiktoken
!pip list | findstr tiktoken

tiktoken                                 0.8.0


In [9]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["AI Accelera has provided Generative AI Training and Consulting Services in more than 100 countries", "Aceleradora AI is the branch of AI Accelera for the Spanish-Speaking market"],
    embedding=OpenAIEmbeddings(),
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

output_parser = StrOutputParser()

C:\Users\Pavel\miniconda3\envs\llms\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [10]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

get_question_and_retrieve_relevant_docs = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

In [11]:
chain = get_question_and_retrieve_relevant_docs | prompt | model | output_parser

chain.invoke("In how many countries has AI Accelera provided services?")

'AI Accelera has provided services in more than 100 countries.'

* Graphic representation of what this chain does:

![alt text](LCEL-chain-graph.png)

## RunnableParallel

In [3]:
#!pip install faiss-cpu

In [4]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["AI Accelera has trained more than 7.000 Alumni from all continents and top companies"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# model = ChatOpenAI(model="gpt-3.5-turbo")
model = ChatOpenAI(model=MODEL_GPT)

retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("who are the Alumni of AI Accelera?")

'The Alumni of AI Accelera are individuals who have been trained by the organization, totaling more than 7,000 from all continents and top companies.'

### Rememeber: Syntax of RunnableParallel can have several variations
* When composing RunnableParallel with another Runnable we do not need to wrap it up in RunnableParallel class. Inside chain, next three syntaxs are equivalent:
    * `RunnableParallel({"context": retriever, "question": RunnablePassthrough()})`
    * `RunnableParallel(context=retriever, question=RunnablePassthrough())`
    * `{"context": retriever, "question": RunnablePassthrough()}`

## Using itemgetter with RunnableParallel

In [5]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# model = ChatOpenAI(model="gpt-3.5-turbo")
model = ChatOpenAI(model=MODEL_GPT)

vectorstore = FAISS.from_texts(
    ["AI Accelera has trained more than 3,000 Enterprise Alumni."], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "How many Enterprise Alumni has trained AI Accelera?", "language": "Pirate English"})

"AI Accelera be havin' trained more than 3,000 Enterprise Alumni, matey! Arrr!"

## RunnablePassthrough

In [6]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    user_input = RunnablePassthrough(),
    transformed_output = lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'user_input': {'num': 1}, 'transformed_output': 2}

## Chaining Runnables

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a sentence about {politician}")

# model = ChatOpenAI(model="gpt-3.5-turbo")
model = ChatOpenAI(model=MODEL_GPT)

chain = prompt | model | StrOutputParser()

In [8]:
chain.invoke("Chamberlain")

'Neville Chamberlain, the British Prime Minister from 1937 to 1940, is best known for his policy of appeasement towards Adolf Hitler, particularly exemplified by the Munich Agreement of 1938.'

### Coercion: Combine chain (which is Runnable) with other Runnables to create new chain

In [9]:
from langchain_core.output_parsers import StrOutputParser

historian_prompt = ChatPromptTemplate.from_template("Was {politician} positive for Humanity?")

composed_chain = {"politician": chain} | historian_prompt | model | StrOutputParser()

In [10]:
composed_chain.invoke({"politician": "Lincoln"})

'Abraham Lincoln is widely regarded as a pivotal figure in American history, particularly for his leadership during the Civil War and his efforts to abolish slavery. Many historians and scholars argue that his actions had a profoundly positive impact on humanity for several reasons:\n\n1. **Abolition of Slavery**: Lincoln\'s issuance of the Emancipation Proclamation in 1863 marked a significant step toward the abolition of slavery in the United States. While it did not immediately free all enslaved people, it paved the way for the eventual passage of the 13th Amendment, which formally abolished slavery.\n\n2. **Preservation of the Union**: Lincoln\'s determination to preserve the Union during the Civil War is seen as crucial for maintaining the integrity of the United States. His leadership helped to prevent the fragmentation of the nation and set the stage for the country\'s future development.\n\n3. **Promotion of Equality**: Lincoln\'s rhetoric and policies increasingly emphasized t

In [11]:
composed_chain.invoke({"politician": "Attila"})

'Attila the Hun is a complex historical figure whose legacy is often viewed through various lenses. On one hand, he was indeed a formidable leader known for his military prowess and ability to unite various tribes under his banner, which allowed him to pose a significant threat to the Roman Empire during the 5th century. His campaigns led to widespread destruction and suffering, causing fear and instability across large parts of Europe.\n\nFrom a historical perspective, some might argue that his leadership led to the eventual transformation of Europe, as the movements of the Huns and the subsequent invasions contributed to the decline of the Western Roman Empire and the rise of new powers and cultures in Europe. This period of upheaval eventually led to the formation of medieval Europe.\n\nHowever, from a humanitarian standpoint, Attila\'s conquests resulted in significant loss of life, displacement, and suffering for many people. His reputation as the "Scourge of God" reflects the dev

### Functions can also be included in Runnables

In [12]:
composed_chain_with_lambda = (
    chain
    | (lambda input: {"politician": input})
    | historian_prompt
    | model
    | StrOutputParser()
)

In [13]:
composed_chain_with_lambda.invoke({"politician": "Robespierre"})

"Maximilien Robespierre is a complex figure in the context of the French Revolution, and opinions about his legacy vary widely. On one hand, he is often seen as a champion of revolutionary ideals such as equality, social justice, and the rights of the common people. His advocacy for the abolition of slavery in the French colonies and his efforts to promote the welfare of the lower classes are viewed by some as progressive and positive contributions to humanity.\n\nOn the other hand, Robespierre is equally known for his role in the Reign of Terror, during which thousands were executed, including many who were deemed enemies of the revolution. This period is characterized by extreme political repression and violence, raising significant moral questions about the methods he employed in the name of revolutionary ideals. Critics argue that his extreme measures undermined the very principles of liberty and justice that the revolution sought to establish.\n\nIn summary, whether Robespierre's 

## Multiple chains

In [14]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt1 = ChatPromptTemplate.from_template("what is the country {politician} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what continent is the country {country} in? respond in {language}"
)

# model = ChatOpenAI()
model = ChatOpenAI(model=MODEL_GPT)

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"country": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke({"politician": "Miterrand", "language": "French"})

'François Mitterrand vient de France, qui est située en Europe.'

## Nested chains

In [15]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables import RunnableParallel

prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

def russian_lastname_from_dictionary(person):
    return person["name"] + "ovich"

chain = RunnableParallel(
    {
        "soccer_player": RunnablePassthrough() 
        | RunnableLambda(russian_lastname_from_dictionary), 
        "operation_c": RunnablePassthrough()
    }
) | prompt | model | output_parser

In [16]:
chain.invoke({
    "name1": "Jordam",
    "name": "Abram"
})

'One curious fact about Roman Abramovich, the Russian businessman and former owner of Chelsea Football Club, is that he has a background in oil and gas. Before making his fortune in the oil industry, Abramovich was an entrepreneur in various businesses, including selling rubber ducks and other toys. His rise to prominence in the oil sector, particularly through his ownership of Sibneft, was a key factor in his wealth accumulation, helping him later to acquire Chelsea in 2003. His diverse early ventures highlight a remarkable transition from small-scale entrepreneurship to becoming one of the most well-known figures in international sports and business.'

## Fallback for chains

In [17]:
# First let's create a chain with a ChatModel
# We add in a string output parser here so the outputs between the two are the same type
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a funny assistant who always includes a joke in your response",
        ),
        ("human", "Who is the best {sport} player worldwide?"),
    ]
)
# Here we're going to use a bad model name to easily create a chain that will error
chat_model = ChatOpenAI(model="gpt-fake")

bad_chain = chat_prompt | chat_model | StrOutputParser()

In [18]:
# Now lets create a chain with the normal OpenAI model
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

prompt_template = """Instructions: You're a funny assistant who always includes a joke in your response.

Question: Who is the best {sport} player worldwide?"""

prompt = PromptTemplate.from_template(prompt_template)

llm = OpenAI()
llm = ChatOpenAI(model=MODEL_GPT)

good_chain = prompt | llm

In [19]:
# We can now create a final chain which combines the two
chain = bad_chain.with_fallbacks([good_chain])

chain.invoke({"sport": "soccer"})

AIMessage(content='Ah, the age-old debate! Some say Lionel Messi, others swear by Cristiano Ronaldo. It’s like trying to decide if pizza or tacos are the best food—both are delicious, but it really depends on your mood! Speaking of soccer, why did the soccer player bring string to the game? Because he wanted to tie the score! 😄', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 32, 'total_tokens': 103, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-1dedf5ae-53c2-4d62-a000-d3a780564300-0', usage_metadata={'input_tokens': 32, 'output_tokens': 71, 'total_tokens': 103, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_detai